# OVERVIEW AG

**ENTRADA**: 
- População Inicial Aleatória
- Função fitness
- Critério de parada (convergência ou n° de gerações)

while criterio de parada:
- 1) aplicar função fitness em cada indivíduo
- 2) Selecionar os melhores indíviduos
- 3) Reprodução
    - Crossover
    - Mutação
- 4) Nova população

**SAÍDA**:
- Melhor indivíviduo da geração final

***PROBLEMA***: Dino Run

- 1) Ações
    - Pular
    - Agachar
    - Não fazer nada

Instalando biblioteca do jogo...

In [1]:
!pip install --user git+https://github.com/GrupoTuringCodes/chrome-trex-rush@master

  Cloning https://github.com/GrupoTuringCodes/chrome-trex-rush (to revision master) to /tmp/pip-req-build-uf_4nwfk
  Running command git clone -q https://github.com/GrupoTuringCodes/chrome-trex-rush /tmp/pip-req-build-uf_4nwfk
  Created wheel for chrome-trex: filename=chrome_trex-0.0.1-py3-none-any.whl size=18064 sha256=13a676c936a70b4d9e8cd5e24c9f44243c2acee8afca47a215428994e7c40f11
  Stored in directory: /tmp/pip-ephem-wheel-cache-qycs7n2b/wheels/ed/54/73/d3d445a03f65e6c50b02a7211fa16a34b7a9f2730413a5c81a
Successfully built chrome-trex


In [2]:
# IMPORTS
import numpy as np
import random
from chrome_trex import DinoGame

In [3]:
# GA VARIABLES
MUTATION_RATE   = .20
CROSSOVER_RATE  = .20
POPULATION_SIZE =  15
ELITISM         =  3  #elitism (steady state)

### Initialize a random population

In [4]:
def initRandomPopulation(size):
    """
    args:
        size: Population Size
    
    output:
        Random Population. Matriz 3x10. Where 3 represents the
        number of possible actions and 10 the weights of each
        action.
    """
    
    population = []
    for i in range(size):
        population.append(np.random.uniform(low=-10, high=10, size=(3,10)))
        
    return population

### Decision Process

In [5]:
def actionValues(individual, state):
    """
    args:
        - individual: 3x10 Matrix with individual weights
        - state: State of the game, represented by an array of 
                 size 10
    
    output:
        - Array with the result of the action in the state analyzed
    """
    return individual @ state
    

In [6]:
def bestPlay(individual, state):
    """
    args:
        - individual: 3x10 Matrix with individual weights
        - state: State of the game, represented by an array of 
                 size 10
    
    output:
        - The action (0, 1 or 2) with best (MAXIMUM) result.
    """
    values = actionValues(individual, state)
    return np.argmax(values)
    

### Reproduction

In [7]:
def mutation(individual):
    """
    args:
        - individual: 3x10 Matrix with individual weights
    output:
    
    """
    
    for i in range(individual.shape[0]): #3
        for j in range(individual.shape[1]): #10
            if np.random.uniform(0, 1) < MUTATION_RATE:
                individual[i][j] *= np.random.uniform(-1.5, 1.5)  
    

In [8]:
def crossover(individual1, individual2):
    """
    args:
        - individual: 3x10 Matrix with individual weights
        
    output:
        - New individual resulting from individual1 and individual2
        combination
    """
    
    child = individual1.copy()
    
    for i in range(individual1.shape[0]): #3
        for j in range(individual1.shape[1]): #10
            if np.random.uniform(0, 1) < CROSSOVER_RATE:
                child[i][j] = individual2[i][j]
    
    return child

### Fitness

In [9]:
def fitness(problem, individual):
    """
    args:
        - problem: object that represents the problem to be solved
        - individual: possible solution
        
    output:
        - score: Value that measure how good the individual performs in
        the problem
    """
    
    problem.reset()
    while not problem.game_over:
        state = problem.get_state()
        action = bestPlay(individual, state)
        problem.step(action)
    
    return problem.get_score()

In [10]:
def sortByFitness(population, order, descending=True):
    """
    args:
        - population: individual to be ordered based on their fitness
        - order: Ordenation priority
        - descending: Boolean variable to decide how to order: ascending 
                      or descending
                      
    output:
    
        - Population Ordered by its fitness
    """
    
    return [x for _, x in sorted(zip(order, population), key=lambda p: p[0], reverse=descending)]


In [11]:
def nextGeneration(population, fitness):
    """
    args:
        - population: list of individuals
        - fitness: list of individual fitness
        
    output:
        - Next generation
        
    STEPS:
        1) ELISTISM
        2) UNTIL FULFIL THE NEXT GENERATION:
            2.1) SELECT RANDOMLY 2 INDIVIDUALS
            2.2) PERFORM CROSSOVER AND GENERATE A NEW INDIVIDUAL
            2.3) PERFORM MUTATIO
            2.4) PERFORM MUTATION
            2.5) APPEND THIS INDIVIDUAL IN THE NEXT GENERATION
    """
    
#     Perform Elitism
    orderedPopulation = sortByFitness(population, fitness)

#   Send the best individuals to the next generation
    nextGeneration = orderedPopulation[:ELITISM]
    
    while len(nextGeneration) < POPULATION_SIZE:
#       Crossover
        ind1, ind2 = random.choices(population, k=2)
        child = crossover(ind1, ind2)
        
#        Mutation
        mutation(child)
        
#        Append individual in the next generation
        nextGeneration.append(child)
    
    return nextGeneration
    
    

# GA Implementation

In [12]:
# Generations
NUM_GENERATIONS = 100

# Problem
problem = DinoGame(fps=50_000)

# Create a random initial population
population = initRandomPopulation(POPULATION_SIZE)

for _ in range(NUM_GENERATIONS):
    
    fitness_list = []
    for ind in population:
        fitness_list.append(fitness(problem, ind))
    
    # Próxima geração
    population = nextGeneration(population, fitness_list)


# Last Generation
fitness_list = []
for ind in population:
    fitness_list.append(fitness(problem, ind))

# Evaluate the best individual
best = (sortByFitness(population, fitness_list))[0]

# Print best individual
print('Best: ', best)

# Execute
fit = fitness(problem, best)
    
print('Fitness: {:4.1f}'.format(fit))

Best:  [[ 6.18209339e-02  1.34365285e-01 -6.95021176e-02 -4.06182574e-01
  -3.17364105e+00 -2.44791037e-01 -4.55713986e-02 -1.58334459e+00
   1.42280139e-01 -8.45957421e-03]
 [-4.26276593e+00  8.23611329e-03  3.14113262e-01  2.34830131e-01
   2.35236303e-01 -3.30818525e-01  1.35009531e-02 -8.32666155e-01
   4.29684988e-02  8.01516444e-02]
 [-8.20767702e-01  8.27226802e-01 -1.15377852e-02 -2.21061990e-01
  -1.12696903e-03  5.07452026e-01 -6.92950583e-01 -2.38577287e-01
  -3.42130255e-01 -2.41693021e+00]]
Fitness: 96.0
